<a href="https://colab.research.google.com/github/maciejlehmann/amw-wdum/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import re
import spacy
from spacy.tokenizer import Tokenizer
from spacy import displacy
from spacy.matcher import Matcher
import textacy

In [77]:
# Czytanie ze stringa

introduction_text = ('To jest tutorial petwarzania naturalnych języków w Spacy.')
introduction_doc = nlp(introduction_text)

print ([token.text for token in introduction_doc])

['To', 'jest', 'tutorial', 'petwarzania', 'naturalnych', 'języków', 'w', 'Spacy', '.']


In [78]:
# Czytanie z pliku

file_name = 'introduction.txt'
introduction_file_text = open(file_name).read()
introduction_file_doc = nlp(introduction_file_text)
# Extract tokens for the given doc
print ([token.text for token in introduction_file_doc])

['To', 'jest', 'tutorial', 'petwarzania', 'naturalnych', 'języków', 'w', 'Spacy', '.']


In [79]:
# Wykrywanie zdań ze stringa

about_text = ('Maciej Lehmann jest studentem 4 roku'
               ' informatyki na Akademii Marynarki'
               ' Wojennej w Gdyni. Jest zainteresowany nauką'
               ' przetwarzania języka naturalnego. Czerwono-niebieski.')
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
len(sentences)

for sentence in sentences:
  print (sentence)

Maciej Lehmann jest studentem 4 roku informatyki na Akademii Marynarki Wojennej w Gdyni.
Jest zainteresowany nauką przetwarzania języka naturalnego.
Czerwono-niebieski.


In [88]:
# Wykrywanie zdań z zastosowaniem własnego separatora

def set_custom_boundaries(doc):
  for token in doc[:-1]:
    if token.text == '...':
     doc[token.i+1].is_sent_start = True
  return doc

ellipsis_text = ('Gus, can you, ... never mind, I forgot'
                  ' what I was saying. So, do you think'
                  ' we should ...')

print("Zdania podzielone przy użyciu nowego separatora: ")
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
  print(sentence)

print("\nZdania z użyciem domyślnego separatora:")
ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
  print(sentence)

Zdania podzielone przy użyciu nowego separatora: 
Gus, can you, ...
never mind, I forgot what I was saying.
So, do you think we should ...

Zdania z użyciem domyślnego separatora:
Gus, can you, ... never mind, I forgot what I was saying.
So, do you think we should ...


In [89]:
# Podział na podstawowe jednostki w tekście - tokeny

for token in about_doc:
  print (token, token.idx)

Maciej 0
Lehmann 7
jest 15
studentem 20
4 30
roku 32
informatyki 37
na 49
Akademii 52
Marynarki 61
Wojennej 71
w 80
Gdyni 82
. 87
Jest 89
zainteresowany 94
nauką 109
przetwarzania 115
języka 129
naturalnego 136
. 147
Czerwono 149
- 157
niebieski 158
. 167


In [90]:
# Wskazanie szczegółowych atrybutów tokenów

for token in about_doc:
  print (token, token.idx, token.text_with_ws, token.is_alpha, token.is_punct, token.is_space, token.shape_, token.is_stop)

Maciej 0 Maciej  True False False Xxxxx False
Lehmann 7 Lehmann  True False False Xxxxx False
jest 15 jest  True False False xxxx False
studentem 20 studentem  True False False xxxx False
4 30 4  False False False d False
roku 32 roku  True False False xxxx False
informatyki 37 informatyki  True False False xxxx False
na 49 na  True False False xx False
Akademii 52 Akademii  True False False Xxxxx False
Marynarki 61 Marynarki  True False False Xxxxx False
Wojennej 71 Wojennej  True False False Xxxxx False
w 80 w  True False False x False
Gdyni 82 Gdyni True False False Xxxxx False
. 87 .  False True False . False
Jest 89 Jest  True False False Xxxx False
zainteresowany 94 zainteresowany  True False False xxxx False
nauką 109 nauką  True False False xxxx False
przetwarzania 115 przetwarzania  True False False xxxx False
języka 129 języka  True False False xxxx False
naturalnego 136 naturalnego True False False xxxx False
. 147 .  False True False . False
Czerwono 149 Czerwono True False

In [91]:
# Dostosowanie procesu tokenizacji, aby wykrywał tokeny na niestandardowych znakach

custom_nlp = spacy.load('en_core_web_sm')
prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
  return Tokenizer(nlp.vocab, prefix_search=prefix_re.search, suffix_search=suffix_re.search, infix_finditer=infix_re.finditer, token_match=None)

custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_about_doc = custom_nlp(about_text)
print([token.text for token in custom_tokenizer_about_doc])

['Maciej', 'Lehmann', 'jest', 'studentem', '4', 'roku', 'informatyki', 'na', 'Akademii', 'Marynarki', 'Wojennej', 'w', 'Gdyni', '.', 'Jest', 'zainteresowany', 'nauką', 'przetwarzania', 'języka', 'naturalnego', '.', 'Czerwono', '-', 'niebieski', '.']


In [102]:
# Wypisanie kilku słów stopów

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
len(spacy_stopwords)

for stop_word in list(spacy_stopwords)[:10]:
  print(stop_word)

whoever
myself
to
neither
every
themselves
fifty
last
see
became


In [99]:
# Wypisanie tokenów pomijając słowa stopy

about_text = ('Gus Proto is a Python developer currently'
               ' working for a London-based Fintech'
               ' company. He is interested in learning'
               ' Natural Language Processing.')
about_doc = nlp(about_text)


for token in about_doc:
  if not token.is_stop:
    print (token)

Gus
Proto
Python
developer
currently
working
London
-
based
Fintech
company
.
interested
learning
Natural
Language
Processing
.


In [100]:
# Tworzenie listy tokenów bez słów stopów

about_no_stopword_doc = [token for token in about_doc if not token.is_stop]
print (about_no_stopword_doc)

[Gus, Proto, Python, developer, currently, working, London, -, based, Fintech, company, ., interested, learning, Natural, Language, Processing, .]


In [104]:
# Redukcja odmioych form wyrazów

conference_help_text = ('Gus is helping organize a developer'
     ' conference on Applications of Natural Language'
     ' Processing. He keeps organizing local Python meetups'
     ' and several internal talks at his workplace.')
conference_help_doc = nlp(conference_help_text)
for token in conference_help_doc:
  print (token, token.lemma_)

Gus Gus
is be
helping help
organize organize
a a
developer developer
conference conference
on on
Applications Applications
of of
Natural Natural
Language Language
Processing Processing
. .
He -PRON-
keeps keep
organizing organize
local local
Python Python
meetups meetup
and and
several several
internal internal
talks talk
at at
his -PRON-
workplace workplace
. .


In [107]:
# Sprawdzanie statystyk występowania słów w danym tekście

from collections import Counter
complete_text = ('Gus Proto is a Python developer currently'
     ' working for a London-based Fintech company. He is'
     ' interested in learning Natural Language Processing.'
     ' There is a developer conference happening on 21 July'
     ' 2019 in London. It is titled "Applications of Natural'
     ' Language Processing". There is a helpline number '
     ' available at +1-1234567891. Gus is helping organize it.'
     ' He keeps organizing local Python meetups and several'
     ' internal talks at his workplace. Gus is also presenting'
     ' a talk. The talk will introduce the reader about "Use'
     ' cases of Natural Language Processing in Fintech".'
     ' Apart from his work, he is very passionate about music.'
     ' Gus is learning to play the Piano. He has enrolled '
     ' himself in the weekend batch of Great Piano Academy.'
     ' Great Piano Academy is situated in Mayfair or the City'
     ' of London and has world-class piano instructors.')

complete_doc = nlp(complete_text)

words = [token.text for token in complete_doc if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

common_words = word_freq.most_common(5)
print (common_words)

unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

words_all = [token.text for token in complete_doc if not token.is_punct]
word_freq_all = Counter(words_all)

common_words_all = word_freq_all.most_common(5)
print (common_words_all)

[('Gus', 4), ('London', 3), ('Natural', 3), ('Language', 3), ('Processing', 3)]
['Proto', 'currently', 'working', 'based', 'company', 'interested', 'conference', 'happening', '21', 'July', '2019', 'titled', 'Applications', 'helpline', 'number', 'available', '+1', '1234567891', 'helping', 'organize', 'keeps', 'organizing', 'local', 'meetups', 'internal', 'talks', 'workplace', 'presenting', 'introduce', 'reader', 'Use', 'cases', 'Apart', 'work', 'passionate', 'music', 'play', 'enrolled', 'weekend', 'batch', 'situated', 'Mayfair', 'City', 'world', 'class', 'piano', 'instructors']
[('is', 10), ('a', 5), ('in', 5), ('Gus', 4), ('of', 4)]


In [117]:
# Określenie jaką część mowy stanowią dane słowa

for token in about_doc:
  print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

nouns = []
adjectives = []
for token in about_doc:
  if token.pos_ == 'NOUN':
    nouns.append(token)
  if token.pos_ == 'ADJ':
    adjectives.append(token)

print("\n\nNouns: ", nouns)

print("Adjectives: ", adjectives)

Gus NNP PROPN noun, proper singular
Proto NNP PROPN noun, proper singular
is VBZ AUX verb, 3rd person singular present
a DT DET determiner
Python NNP PROPN noun, proper singular
developer NN NOUN noun, singular or mass
currently RB ADV adverb
working VBG VERB verb, gerund or present participle
for IN ADP conjunction, subordinating or preposition
a DT DET determiner
London NNP PROPN noun, proper singular
- HYPH PUNCT punctuation mark, hyphen
based VBN VERB verb, past participle
Fintech NNP PROPN noun, proper singular
company NN NOUN noun, singular or mass
. . PUNCT punctuation mark, sentence closer
He PRP PRON pronoun, personal
is VBZ AUX verb, 3rd person singular present
interested JJ ADJ adjective
in IN ADP conjunction, subordinating or preposition
learning VBG VERB verb, gerund or present participle
Natural NNP PROPN noun, proper singular
Language NNP PROPN noun, proper singular
Processing NNP PROPN noun, proper singular
. . PUNCT punctuation mark, sentence closer


Nouns:  [develope

In [120]:
about_interest_text = ('He is interested in learning' ' Natural Language Processing.')
about_interest_doc = nlp(about_interest_text)
displacy.render(about_interest_doc, style='dep', jupyter=True)

In [122]:
# Tworzenie funkcji, która konwertuje tekst do formy, którą można analizować

def is_token_allowed(token):
  if (not token or not token.string.strip() or
    token.is_stop or token.is_punct):
    return False
  return True

def preprocess_token(token):
  return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)
for token in complete_doc if is_token_allowed(token)]
complete_filtered_tokens

['gus',
 'proto',
 'python',
 'developer',
 'currently',
 'work',
 'london',
 'base',
 'fintech',
 'company',
 'interested',
 'learn',
 'natural',
 'language',
 'processing',
 'developer',
 'conference',
 'happen',
 '21',
 'july',
 '2019',
 'london',
 'title',
 'applications',
 'natural',
 'language',
 'processing',
 'helpline',
 'number',
 'available',
 '+1',
 '1234567891',
 'gus',
 'help',
 'organize',
 'keep',
 'organize',
 'local',
 'python',
 'meetup',
 'internal',
 'talk',
 'workplace',
 'gus',
 'present',
 'talk',
 'talk',
 'introduce',
 'reader',
 'use',
 'case',
 'natural',
 'language',
 'processing',
 'fintech',
 'apart',
 'work',
 'passionate',
 'music',
 'gus',
 'learn',
 'play',
 'piano',
 'enrol',
 'weekend',
 'batch',
 'great',
 'piano',
 'academy',
 'great',
 'piano',
 'academy',
 'situate',
 'mayfair',
 'city',
 'london',
 'world',
 'class',
 'piano',
 'instructor']

In [126]:
# Wyciąganie imienia i nazwiska za pomocą dopasowania opartego o wskazane reguły

matcher = Matcher(nlp.vocab)
def extract_full_name(nlp_doc):
  pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
  matcher.add('FULL_NAME', None, pattern)
  matches = matcher(nlp_doc)
  for match_id, start, end in matches:
    span = nlp_doc[start:end]
    return span.text

extract_full_name(about_doc)

'Gus Proto'

In [127]:
# Użycie dopasowania opartego na regułach do wyciągnięcia numeru telefonu

matcher = Matcher(nlp.vocab)
conference_org_text = ('There is a developer conference'
     'happening on 21 July 2019 in London. It is titled'
     ' "Applications of Natural Language Processing".'
     ' There is a helpline number available'
     ' at (123) 456-789')

def extract_phone_number(nlp_doc):
  pattern = [{'ORTH': '('}, {'SHAPE': 'ddd'},
                {'ORTH': ')'}, {'SHAPE': 'ddd'},
                {'ORTH': '-', 'OP': '?'},
                {'SHAPE': 'ddd'}]
  matcher.add('PHONE_NUMBER', None, pattern)
  matches = matcher(nlp_doc)
  for match_id, start, end in matches:
    span = nlp_doc[start:end]
    return span.text

conference_org_doc = nlp(conference_org_text)
extract_phone_number(conference_org_doc)

'(123) 456-789'

In [129]:
piano_text = 'Gus is learning piano'
piano_doc = nlp(piano_text)
for token in piano_doc:
  print (token.text, token.tag_, token.head.text, token.dep_)

displacy.render(piano_doc, style='dep', jupyter=True)

Gus NNP learning nsubj
is VBZ learning aux
learning VBG learning ROOT
piano NN learning dobj


In [130]:
# Drzewo posiada informacje o strukturze zdań i gramatyce i można je przeglądać na różne sposoby, aby wyodrębnić relacje

one_line_about_text = ('Gus Proto is a Python developer'
' currently working for a London-based Fintech company')
one_line_about_doc = nlp(one_line_about_text)
# Extract children of `developer`
print([token.text for token in one_line_about_doc[5].children])

# Extract previous neighboring node of `developer`
print (one_line_about_doc[5].nbor(-1))

# Extract next neighboring node of `developer`
print (one_line_about_doc[5].nbor())

# Extract all tokens on the left of `developer`
print([token.text for token in one_line_about_doc[5].lefts])

# Extract tokens on the right of `developer`
print([token.text for token in one_line_about_doc[5].rights])

# Print subtree of `developer`
print (list(one_line_about_doc[5].subtree))

['a', 'Python', 'working']
Python
currently
['a', 'Python']
['working']
[a, Python, developer, currently, working, for, a, London, -, based, Fintech, company]


In [131]:
# Tworzenie funkcji, która tworzy stringa na podstawie drzewa

def flatten_tree(tree):
  return ''.join([token.text_with_ws for token in list(tree)]).strip()

# Print flattened subtree of `developer`
print (flatten_tree(one_line_about_doc[5].subtree))

a Python developer currently working for a London-based Fintech company


In [132]:
# Wykrywanie fraz rzeczownikowych w tekście

conference_text = ('There is a developer conference' ' happening on 21 July 2019 in London.')
conference_doc = nlp(conference_text)
# Extract Noun Phrases
for chunk in conference_doc.noun_chunks:
  print (chunk)

a developer conference
21 July
London


In [137]:
# Wykrywanie fraz czaskownikowych w tekście

about_talk_text = ('The talk will introduce reader about Use'
                    ' cases of Natural Language Processing in'
                    ' Fintech')
pattern = r'(<VERB>?<ADV>*<VERB>+)'
about_talk_doc = textacy.make_spacy_doc(about_talk_text, lang='en_core_web_sm')
verb_phrases = textacy.extract.pos_regex_matches(about_talk_doc, pattern)
# Print all Verb Phrase
for chunk in verb_phrases:
  print(chunk.text)

# Extract Noun Phrase to explain what nouns are involved
for chunk in about_talk_doc.noun_chunks:
  print (chunk)

will introduce
The talk
reader
Use cases
Natural Language Processing
Fintech


/usr/local/lib/python3.6/dist-packages/textacy/extract.py:338: DeprecationWarning: `pos_regex_matches()` has been deprecated! for similar but more powerful and performant functionality, use `textacy.extract.matches()` instead.
  action="once",


In [139]:
# Znajdowanie nazwanych obiektów w tekście

piano_class_text = ('Great Piano Academy is situated'
     ' in Mayfair or the City of London and has'
     ' world-class piano instructors.')
piano_class_doc = nlp(piano_class_text)
for ent in piano_class_doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_, spacy.explain(ent.label_))

displacy.render(piano_class_doc, style='ent', jupyter=True)

Great Piano Academy 0 19 ORG Companies, agencies, institutions, etc.
Mayfair 35 42 GPE Countries, cities, states
the City of London 46 64 GPE Countries, cities, states


In [141]:
# Ukrywanie imion w tekście w oparciu o nazwane obiekty

survey_text = ('Out of 5 people surveyed, James Robert,'
                ' Julie Fuller and Benjamin Brooks like'
                ' apples. Kelly Cox and Matthew Evans'
                ' like oranges.')

def replace_person_names(token):
  if token.ent_iob != 0 and token.ent_type_ == 'PERSON':
    return '[CONFIDENTIAL] '
  return token.string

def redact_names(nlp_doc):
  for ent in nlp_doc.ents:
    ent.merge()
  tokens = map(replace_person_names, nlp_doc)
  return ''.join(tokens)

survey_doc = nlp(survey_text)
redact_names(survey_doc)

'Out of 5 people surveyed, [CONFIDENTIAL] , [CONFIDENTIAL] and [CONFIDENTIAL] like apples. [CONFIDENTIAL] and [CONFIDENTIAL] like oranges.'